# Combine price files on `measure_date_CET` and add UTC timestamp

In [34]:
import pandas as pd
from zoneinfo import ZoneInfo

# File paths for input data
DA_PATH = "/home/renga/Desktop/neoen_data/renga_work/data/grid_penalty/da_prices.csv"
PRE_PATH = "/home/renga/Desktop/neoen_data/renga_work/data/grid_penalty/balancing_energy_imbalance_data(in).csv"

# Output file path
OUTPUT_CSV = "/home/renga/Desktop/neoen_data/renga_work/data/grid_penalty/combined_with_imbalance.csv"


In [35]:

da = pd.read_csv(DA_PATH)
pre = pd.read_csv(PRE_PATH)

print("DA shape:", da.shape)
print("PRE shape:", pre.shape)

# Quick peek
display(da.head(3))
display(pre.head(3))


DA shape: (8376, 2)
PRE shape: (8545, 10)


,measure_date,da_price
0,2025-01-01 00:00:00+01:00,12.36
1,2025-01-01 01:00:00+01:00,18.92
2,2025-01-01 02:00:00+01:00,16.66


,measure_date,end_date,resolution,imbalance,system_trend,positive_imbalance_settlement_price,negative_imbalance_settlement_price,missing_data_list,updated_date,partition_day
0,2025-07-06 22:00:00.0000000 +00:00,2025-07-06 22:15:00.0000000 +00:00,PT15M,-128,HAUSSE,42.61,46.15,NaN,2025-07-14 12:28:04.0000000 +00:00,20250707
1,2025-07-06 22:15:00.0000000 +00:00,2025-07-06 22:30:00.0000000 +00:00,PT15M,-104,HAUSSE,39.36,42.62,NaN,2025-07-14 12:28:04.0000000 +00:00,20250707
2,2025-07-06 22:30:00.0000000 +00:00,2025-07-06 22:45:00.0000000 +00:00,PT15M,27,BAISSE,-30.10,-27.80,NaN,2025-07-08 08:30:15.0000000 +00:00,20250707


In [37]:
# --- Ensure the join key exists in both ---
KEY = "measure_date"
assert KEY in da.columns, f"'{KEY}' not found in DA file columns: {list(da.columns)}"
assert KEY in pre.columns, f"'{KEY}' not found in PRE file columns: {list(pre.columns)}"



In [39]:
# --- Robust parse of 'measure_date_CET' handling offset-aware strings (+HH:MM / Z) and naive ones ---

import pandas as pd
from pandas.api.types import is_datetime64_any_dtype

KEY = "measure_date"
PARIS_TZ = "Europe/Paris"

def to_paris_tzaware(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.strip()

    # Identify strings that already have an explicit offset or 'Z'
    # e.g., '2025-03-30 03:00:00+02:00' or '2025-03-30T01:00:00Z'
    mask_offset = s.str.contains(r"([+-]\d{2}:\d{2}|Z)$", na=False, regex=True)

    # Prepare an output series with the target tz-aware dtype
    out = pd.Series(pd.NaT, index=s.index, dtype=f"datetime64[ns, {PARIS_TZ}]")

    # offset-aware strings -> parse with utc=True, then convert to Paris
    if mask_offset.any():
        dt_off = pd.to_datetime(s[mask_offset], errors="coerce", utc=True)
        # Convert UTC -> Europe/Paris
        dt_off_paris = dt_off.dt.tz_convert(PARIS_TZ)
        out.loc[mask_offset] = dt_off_paris

    # naive strings (no offset, no Z) -> parse and localize to Paris
    if (~mask_offset).any():
        s_naive = s[~mask_offset]

        # Clean tokens like 'CET'/'CEST' if present 
        s_naive = s_naive.str.replace(r"\s*(CET|CEST)$", "", regex=True, case=False).str.strip()

        # Try parsing as month-first, then day-first
        dt1 = pd.to_datetime(s_naive, errors="coerce", dayfirst=False)
        dt2 = pd.to_datetime(s_naive, errors="coerce", dayfirst=True)
        dt_naive = dt1.fillna(dt2)

        if not is_datetime64_any_dtype(dt_naive):
            raise TypeError(f"Could not parse '{KEY}' to datetimes (naive branch).")

        # Localize to Europe/Paris (handles DST)
        dt_naive_paris = dt_naive.dt.tz_localize(PARIS_TZ, ambiguous="infer", nonexistent="shift_forward")
        out.loc[~mask_offset] = dt_naive_paris

    return out

# Build tz-aware helper columns on both dataframes
da["_measure_dt_paris"] = to_paris_tzaware(da[KEY])
pre["_measure_dt_paris"] = to_paris_tzaware(pre[KEY])

print("da _measure_dt_paris dtype:", da["_measure_dt_paris"].dtype)
print("pre _measure_dt_paris dtype:", pre["_measure_dt_paris"].dtype)


/tmp/ipykernel_143544/324459687.py:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_offset = s.str.contains(r"([+-]\d{2}:\d{2}|Z)$", na=False, regex=True)
/tmp/ipykernel_143544/324459687.py:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_offset = s.str.contains(r"([+-]\d{2}:\d{2}|Z)$", na=False, regex=True)


da _measure_dt_paris dtype: datetime64[ns, Europe/Paris]
pre _measure_dt_paris dtype: datetime64[ns, Europe/Paris]


In [40]:
# --- Merge on the original CET key (naive) ---
# We merge on the naive datetime values to align rows as they appear in your inputs.
# If you'd rather merge on the tz-aware version, switch 'on=KEY' to 'left_on="_measure_dt_paris", right_on="_measure_dt_paris"'.
combined = pd.merge(
    da, pre, on=KEY, how="outer", suffixes=("_da", "_pre")
).sort_values(KEY).reset_index(drop=True)

print("Combined shape:", combined.shape)
combined.head(5)


Combined shape: (16921, 13)


,measure_date,da_price,_measure_dt_paris_da,end_date,resolution,imbalance,system_trend,positive_imbalance_settlement_price,negative_imbalance_settlement_price,missing_data_list,updated_date,partition_day,_measure_dt_paris_pre
0,2025-01-01 00:00:00+01:00,12.36,2025-01-01 00:00:00+01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,2025-01-01 01:00:00+01:00,18.92,2025-01-01 01:00:00+01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,2025-01-01 02:00:00+01:00,16.66,2025-01-01 02:00:00+01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,2025-01-01 03:00:00+01:00,13.10,2025-01-01 03:00:00+01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,2025-01-01 04:00:00+01:00,5.90,2025-01-01 04:00:00+01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [41]:
# --- Build the UTC column ---
# Prefer the tz-aware time from DA if present, else from PRE.
def pick_paris_time(row):
    val = row.get("_measure_dt_paris_da")
    if pd.isna(val):
        return row.get("_measure_dt_paris_pre")
    return val

# Ensure both helper columns exist in 'combined':
# '_measure_dt_paris_da' and '_measure_dt_paris_pre'
# Let's ensure they exist (if one side didn't have rows, pandas may fill with NaT).
if "_measure_dt_paris_da" not in combined.columns:
    combined["_measure_dt_paris_da"] = pd.NaT
if "_measure_dt_paris_pre" not in combined.columns:
    combined["_measure_dt_paris_pre"] = pd.NaT

combined["_measure_dt_paris"] = combined.apply(pick_paris_time, axis=1)

# Convert to UTC
combined["measure_date_UTC"] = combined["_measure_dt_paris"].dt.tz_convert("UTC")

# Optional: convenience string version like '2025-10-08 11:15:00+00:00'
combined["measure_date_UTC_str"] = combined["measure_date_UTC"].dt.strftime("%Y-%m-%d %H:%M:%S%z")
# Insert the colon in the +00:00 offset to match the exact example format
combined["measure_date_UTC_str"] = combined["measure_date_UTC_str"].str.replace(r"(\+\d{2})(\d{2})$", r"\1:\2", regex=True)

# Reorder columns to place the new UTC columns next to the CET key
cols = list(combined.columns)
for c in ["measure_date_UTC", "measure_date_UTC_str"]:
    cols.insert(cols.index(KEY) + 1, cols.pop(cols.index(c)))
combined = combined[cols]

combined.head()


,measure_date,measure_date_UTC_str,measure_date_UTC,da_price,_measure_dt_paris_da,end_date,resolution,imbalance,system_trend,positive_imbalance_settlement_price,negative_imbalance_settlement_price,missing_data_list,updated_date,partition_day,_measure_dt_paris_pre,_measure_dt_paris
0,2025-01-01 00:00:00+01:00,2024-12-31 23:00:00+00:00,2024-12-31 23:00:00+00:00,12.36,2025-01-01 00:00:00+01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2025-01-01 00:00:00+01:00
1,2025-01-01 01:00:00+01:00,2025-01-01 00:00:00+00:00,2025-01-01 00:00:00+00:00,18.92,2025-01-01 01:00:00+01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2025-01-01 01:00:00+01:00
2,2025-01-01 02:00:00+01:00,2025-01-01 01:00:00+00:00,2025-01-01 01:00:00+00:00,16.66,2025-01-01 02:00:00+01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2025-01-01 02:00:00+01:00
3,2025-01-01 03:00:00+01:00,2025-01-01 02:00:00+00:00,2025-01-01 02:00:00+00:00,13.10,2025-01-01 03:00:00+01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2025-01-01 03:00:00+01:00
4,2025-01-01 04:00:00+01:00,2025-01-01 03:00:00+00:00,2025-01-01 03:00:00+00:00,5.90,2025-01-01 04:00:00+01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2025-01-01 04:00:00+01:00


In [42]:
# --- Expand Day-Ahead (hourly) price to 15-minute values using the UTC timestamp ---


import numpy as np
from pandas.api.types import is_numeric_dtype

# Identify candidate DA columns (adjust list if you have custom names)
candidate_names = {"da_price", "price_da", "day_ahead", "day_ahead_price", "da"}
numeric_cols = [c for c in combined.columns if is_numeric_dtype(combined[c])]

da_cols = [c for c in numeric_cols if c.endswith("_da") or c.lower() in candidate_names]

if not da_cols:
    raise ValueError(
        "No Day-Ahead (DA) price column detected. "
        "Rename your DA column to end with '_da' or to one of: "
        f"{sorted(candidate_names)}"
    )

# Use UTC to avoid DST pitfalls during resampling/filling
if "measure_date_UTC" not in combined.columns:
    raise KeyError("measure_date_UTC column not found. Create it before running this cell.")

# Floor each row to the top of the hour in UTC
combined["_hour_utc"] = combined["measure_date_UTC"].dt.floor("H")

# For each DA column, map the hourly :00 price to all 15-min rows in that hour
for col in da_cols:
    # Build a map from hour -> DA price taken from the :00 minute rows
    mask_on_the_hour = combined["measure_date_UTC"].dt.minute == 0
    hour_price_map = (
        combined.loc[mask_on_the_hour, ["_hour_utc", col]]
                .dropna()
                .groupby("_hour_utc")[col]
                .last()   # in case duplicates, take the last observed for that hour
    )

    # Create the 15-min expanded column
    out_col = f"{col}_15min"
    combined[out_col] = combined["_hour_utc"].map(hour_price_map)

    # Optional: if some hours are missing a :00 value, you can derive from nearest hour by forward/back-fill:
    # combined[out_col] = (
    #     combined.set_index("measure_date_UTC")[out_col]
    #             .sort_index()
    #             .ffill()
    #             .bfill()
    #             .reindex(combined["measure_date_UTC"])
    #     .values
    # )

# (leave _hour_utc for inspection; it will be dropped in the save/cleanup cell)
print("Created 15-min expanded columns for:", da_cols)


Created 15-min expanded columns for: ['da_price']


/tmp/ipykernel_143544/832121445.py:25: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  combined["_hour_utc"] = combined["measure_date_UTC"].dt.floor("H")


In [43]:
# --- Clean up helper columns & save ---
drop_helpers = [c for c in combined.columns if c.startswith("_measure_dt_paris")]
combined_out = combined.drop(columns=drop_helpers)



In [44]:
# --- Drop unwanted columns and save the cleaned CSV ---

from pathlib import Path

DROP_COLS = [
    "da_price",
    "end_date",
    "resolution",
    "system_trend",
    "missing_data_list",
    "updated_date",
    "partition_day",
    "_hour_utc",
    "measure_date_CET",
    "measure_date_UTC_str",
    # "da_price_15min"
]

# Ensure the combined_out object exists
if 'combined_out' not in globals():
    raise NameError("combined_out is not defined. Run the earlier cells that produce 'combined_out' before running this cell.")

# Quick preview of columns before dropping
print("Columns currently in combined_out:")
print(combined_out.columns.tolist())

# Drop the specified columns (ignore any names that don't exist)
if DROP_COLS:
    combined_out = combined_out.drop(columns=DROP_COLS, errors='ignore')
    print(f"Dropped columns (requested): {DROP_COLS}")
else:
    print("No DROP_COLS provided; nothing dropped.")


Columns currently in combined_out:
['measure_date', 'measure_date_UTC_str', 'measure_date_UTC', 'da_price', 'end_date', 'resolution', 'imbalance', 'system_trend', 'positive_imbalance_settlement_price', 'negative_imbalance_settlement_price', 'missing_data_list', 'updated_date', 'partition_day', '_hour_utc', 'da_price_15min']
Dropped columns (requested): ['da_price', 'end_date', 'resolution', 'system_trend', 'missing_data_list', 'updated_date', 'partition_day', '_hour_utc', 'measure_date_CET', 'measure_date_UTC_str']


In [45]:
# Identify an imbalance column from common name variants (case-insensitive)
IMBALANCE_CANDIDATES = ['imbalance']

cols_lower = {c.lower(): c for c in combined_out.columns}
imbalance_col = None
for cand in IMBALANCE_CANDIDATES:
    if cand in cols_lower:
        imbalance_col = cols_lower[cand]
        break

print("Rows before drop:", len(combined_out))

# If found, drop rows where imbalance is missing/empty
if imbalance_col is not None:
    # Treat empty strings and 'nan' strings as missing too
    s = combined_out[imbalance_col].astype(str)
    mask_missing = combined_out[imbalance_col].isna() | s.str.strip().eq("") | s.str.lower().eq("nan")
    missing_count = int(mask_missing.sum())
    if missing_count:
        combined_out = combined_out.loc[~mask_missing].reset_index(drop=True)
        print(f"Dropped {missing_count} rows with missing '{imbalance_col}'")
    else:
        print(f"No missing values found in '{imbalance_col}'")
else:
    print("No imbalance-like column found; skipping row-drop step.")


# Ensure output path exists and save
if 'OUTPUT_CSV' not in globals():
    raise NameError("OUTPUT_CSV is not defined. Define OUTPUT_CSV earlier in the notebook (the destination path).")

out_path = Path(OUTPUT_CSV)
out_path.parent.mkdir(parents=True, exist_ok=True)
combined_out.to_csv(out_path, index=False)
print(f"Saved {len(combined_out)} rows to {out_path}")

Rows before drop: 16921
Dropped 8376 rows with missing 'imbalance'
Saved 8545 rows to /home/renga/Desktop/neoen_data/renga_work/data/grid_penalty/combined_with_imbalance.csv
